# Laboratório 3: Classificando textos com a base de dados IMDB

Bem vindo ao laboratório 3 do Mini curso de TensorFlow, neste tutorial vamos aprender a classificar avaliacoes de filmes utilizando o TensorFlow 2 e o keras.

<img src="imgs/tf_logo_social.png" width="500">

O objetivo deste laboratorio será classificar avaliações de filmes em **positivas** ou **negativas** utilizando uma rede neural que construiremos no decorrer deste tutorial e um bastante aplicado tipo de problema de aprendizado de máquina conhecido como classificação binária. Para isso, utilizaremos a base de dados do site IMDB (Internet Movie Database) que possui avaliaçòes de mais de 50000 filmes. 

## Importando as bibliotecas necessárias para o projeto

Para realizarmos a construção da nossa rede neural com o Keras, precisamos importar algumas bibliotecas essenciais para a construção da nossa rede e algumas outras bibliotecas que nos ajudarão a ter uma melhor visualização e manipulação dos dados. O código abaixo realiza as importações necessárias, não se preocupe pois as respectivas bibliotecas já se encontram devidamente instaladas:

In [17]:
# Importando o TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras

# Bibliotecas auxiliares
import numpy as np
import matplotlib.pyplot as plt

# Verificamos se a importação foi realizada
print(tf.__version__)

1.14.0


Caso tenha sido impresso a versão do Tensorflow instalada, a importação foi feita com sucesso e o próximo passo será conhecer melhor o nosso dataset.

## Conhecendo o dataset

Utilizaremos como dataset a base de dados **IMBD** que está disponível junto ao TensorFlow, esta base de dados se encontra pré-processada de forma que as avaliações (ou sequências de palavras) foram convertidas em sequências de inteiros, onde cada inteiro representa uma palavra específica no dicionário. Utilizaremos o método *load_data()* para carregar os dados e separá-los em dois grupos: um grupo para testes com 25.000 avaliações e um grupo para treinamento com também 25.000 avaliações.

In [18]:
dataset_imdb = keras.datasets.imdb

(dados_treinamento, dados_treinamento_labels), (dados_teste, dados_teste) = dataset_imdb.load_data(num_words=10000)

Observe que definimos um parâmetro no método *load_data()*  conhecido como *num_words*, este parâmetro em nosso exemplo indica que queremos obter as 10000 palavras mais frequentes presentes no conjunto de treinamento. As palavras menos frequêntes não possuem muita relevância para o treinamento e seu uso pode aumentar a complexidade da rede e prejudicar o treinamento, por este fato estas palavras serão descartadas. Vamos primeiro imprimir tamanho total de avaliações presente nos conjuntos de teste e treinamento:

In [19]:
print("Tamanho do conjunto de Treinamento: {}".format(len(dados_treinamento)))

print("Tamanho do conjunto de Teste: {}".format(len(dados_teste)))

Tamanho do conjunto de Treinamento: 25000
Tamanho do conjunto de Teste: 25000


Como podemos observar, temos cerca de 25.000 avaliações para cada conjunto (teste e treinamento). Os *labels* correspondem a um valor inteiro com valor de 0 ou 1, onde 0 é uma avaliação negativa e 1 respresenta uma avaliação positiva. Para conhecermos mais sobre o nosso dataset, vamos visualizar a primeira avaliação presente no conjunto de treinamento:

In [20]:
print(dados_treinamento[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


A saída acima nos mostra uma sequência de números presentes em uma lista, cada número desse representa uma palavra dentro do vocabulário conhecido em nossa base de dados. O tamanho do vocabulário existente em nosso dataset será de 10.000 palavras, que são as palavras de maior utilização e consequêntemente maior relevância para o treinamento. Mas simplesmente dizer que esta sequência de números corresponde a uma avaliação de algum filme não é aceitável, precisamos também construir um método para traduzir esses valores em palavras novamente e assim podermos visualizar melhor os dados em seu estado original. Primeiro precisamo carregar o vocabulário para depois podermos começar a trazuir as avaliações:

In [33]:
palavras_vocabulario = dataset_imdb.get_word_index()

print("Número total de palavras: {} | Tipo: {}".format(len(palavras_vocabulario),type(palavras_vocabulario)))
# print(palavras_vocabulario)

Número total de palavras: 88584 | Tipo: <class 'dict'>


Em seguida precisamos formartar o nosso vocabulário, adicionando mais alguns valores que representam aspectos do texto como ponto de inicio e caracter desconhecido para o vocabulário.

In [42]:

palavras_vocabulario = {k:(v+3) for k,v in palavras_vocabulario.items()}
palavras_vocabulario["<PAD>"] = 0 # preenchimento (mais a frente fará sentido)
palavras_vocabulario["<START>"] = 1 # inicio do texto
palavras_vocabulario["<UNK>"] = 2  # caractere desconhecido (unknown)
palavras_vocabulario["<UNUSED>"] = 3 # caractere não utilizado


Finalizamos a formatação invertendo os valores das chaves com seus respectivos valores inteiros, basicamente o que antes era *{"TensorFlow": 200}* se transforma em *{200 : "TensorFlow"}*. Esta manipulação nos permitirá manipular melhor o vocabulário, "revertendo" as posições de valores: 

In [43]:
palavras_vocabulario_reverso = dict([(value, key) for (key, value) in palavras_vocabulario.items()])

É válido destacar que a variável *palavras_vocabulario_reverso* somente terá utilizada na tradução dos valores inteiros para palavras. Finalizamos a parte de tradução, criando um metodo chamado *decodifica_avaliacao()* que ficará responsável por traduzir os valores para palavras.

In [46]:
def decodifica_avaliacao(texto):
    return ' '.join([palavras_vocabulario_reverso.get(i, '?') for i in texto])

Finalmente podemos verificar se o nosso método *decodifica_avaliacao()* é capaz de traduzir perfeitamente as avaliações e assim nos permitir visualizá-las para conhecer melhor o dataset. Vamos imprimir inicialmente a primeira avaliação contida no conjunto de treinamento e em seguida exibiremos a sua respectiva tradução:

In [47]:
print("Avaliação original: {}".format(dados_treinamento[0]))

print("\nAvaliação traduzida: {}".format(decodifica_avaliacao(dados_treinamento[0])))

Avaliação original: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]

Avaliação traduzida: <START> this film 

## Preparando os dados para o treinamento